In [5]:
from nbformat import v4 as nbf
from pathlib import Path
import nbformat as nbfmt

cells=[]

cells.append(nbf.new_markdown_cell("""# 🔗 비지도 학습 파이프라인 가이드
- **목표**: 결측/스케일/인코딩 → (선택)차원축소 → **군집** → 내부지표 평가까지 한 번에
- **포인트**: 지도와 같은 `Pipeline`을 쓰되, **평가는 Silhouette 등 내부지표**로!
"""))

cells.append(nbf.new_code_cell("""# 폰트(선택): 한글 그래프용
import matplotlib.pyplot as plt, platform
if platform.system() == 'Linux':
    plt.rcParams['font.family'] = 'NanumGothic'
elif platform.system() == 'Darwin':
    plt.rcParams['font.family'] = 'AppleGothic'
else:
    plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False"""))

cells.append(nbf.new_code_cell("""# 라이브러리
import numpy as np, pandas as pd, seaborn as sns
from sklearn.datasets import load_iris
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, Normalizer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.pipeline import Pipeline
from sklearn.metrics import silhouette_score
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt

# 데이터: iris (수치형만 있음)
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
X.head()"""))

cells.append(nbf.new_markdown_cell("""## 🧱 1) 기본 파이프라인 (결측 → 스케일 → KMeans)
- 비지도도 **같은 Pipeline** 사용
- 내부지표로 silhouette 계산"""))

cells.append(nbf.new_code_cell("""# 결측치 대응(데모용, iris는 결측 거의 없음)
imputer = SimpleImputer(strategy='median')
scaler = StandardScaler()

pipe_kmeans = Pipeline([
    ("impute", imputer),
    ("scale", scaler),
    ("cluster", KMeans(n_clusters=3, n_init=10, random_state=42))
])

labels = pipe_kmeans.fit_predict(X)  # 비지도는 fit_predict가 편함
sil = silhouette_score(X=StandardScaler().fit_transform(X), labels=labels)
print(f"Silhouette (k=3): {sil:.3f}")"""))

cells.append(nbf.new_markdown_cell("""## 🎯 2) k 탐색 (수동 루프) — Silhouette 최대화"""))

cells.append(nbf.new_code_cell("""Ks = range(2,11)
scores = []
for k in Ks:
    pipe = Pipeline([
        ("impute", SimpleImputer(strategy='median')),
        ("scale", StandardScaler()),
        ("cluster", KMeans(n_clusters=k, n_init=10, random_state=42))
    ])
    lab = pipe.fit_predict(X)
    # 같은 스케일 기준으로 실루엣 계산
    sil = silhouette_score(StandardScaler().fit_transform(X), lab)
    scores.append(sil)

plt.figure(figsize=(6,4))
plt.plot(list(Ks), scores, marker='o')
plt.title("Silhouette vs k (KMeans)")
plt.xlabel("k"); plt.ylabel("silhouette")
plt.grid(True, linestyle='--', alpha=0.4)
plt.show()

best_k = list(Ks)[int(np.argmax(scores))]
print("Best k by silhouette:", best_k)"""))

cells.append(nbf.new_markdown_cell("""## 🧭 3) (선택) PCA 포함 파이프라인
- 고차원/노이즈가 있으면 **PCA로 안정화** 가능
- 시각화(2D)도 용이"""))

cells.append(nbf.new_code_cell("""pipe_kmeans_pca = Pipeline([
    ("impute", SimpleImputer(strategy='median')),
    ("scale", StandardScaler()),
    ("pca", PCA(n_components=2, random_state=42)),
    ("cluster", KMeans(n_clusters=3, n_init=10, random_state=42))
])

lab_pca = pipe_kmeans_pca.fit_predict(X)
Xp = pipe_kmeans_pca.named_steps["pca"].transform(
    StandardScaler().fit_transform(SimpleImputer(strategy='median').fit_transform(X))
)

plt.figure(figsize=(6,5))
plt.scatter(Xp[:,0], Xp[:,1], c=lab_pca, cmap='viridis', s=50, alpha=0.8, edgecolor='k')
plt.title("KMeans + PCA (Iris, 2D)")
plt.xlabel("PC1"); plt.ylabel("PC2")
plt.grid(True, linestyle='--', alpha=0.3)
plt.show()"""))

cells.append(nbf.new_markdown_cell("""## 📌 4) DBSCAN 파이프라인 + eps 탐색 힌트
- 밀도 기반: **eps/min_samples** 중요
- 스케일에 민감 → **스케일링 필수**
- 내부지표는 노이즈(-1) 있을 때 주의해서 해석"""))

cells.append(nbf.new_code_cell("""def run_dbscan(eps=0.6, min_samples=5):
    pipe_db = Pipeline([
        ("impute", SimpleImputer(strategy='median')),
        ("scale", StandardScaler()),
        ("cluster", DBSCAN(eps=eps, min_samples=min_samples))
    ])
    labels = pipe_db.fit_predict(X)  # -1은 노이즈
    # 모든 점이 -1이면 silhouette 계산 불가 → 예외 처리
    if len(set(labels))<=1 or (set(labels)=={-1}):
        print(f"DBSCAN eps={eps}: all noise → skip silhouette")
        return labels, None
    sil = silhouette_score(StandardScaler().fit_transform(X), labels)
    print(f"DBSCAN eps={eps} silhouette: {sil:.3f} (clusters={len(set(labels))- (1 if -1 in labels else 0)})")
    return labels, sil

labels_db, sil_db = run_dbscan(eps=0.6, min_samples=5)"""))

cells.append(nbf.new_markdown_cell("""## 👀 5) 정리
- 비지도도 **Pipeline** 그대로 쓴다 ✅  
- 차이는 **평가/튜닝 방식**: 내부지표(Silhouette 등)와 수동/커스텀 탐색  
- 데이터 특성에 맞춘 전처리(스케일/차원축소/이상치)가 성공의 핵심"""))

nb = nbf.new_notebook(cells=cells)
out_dir = Path("_ml_dl_입문_실습_250804/06_비지도_학습/00_로드맵")
out_dir.mkdir(parents=True, exist_ok=True)
out_path = out_dir / "6-0_비지도_파이프라인_가이드.ipynb"
with open(out_path, "w", encoding="utf-8") as f:
    nbfmt.write(nb, f)
print(f"[OK] 저장 완료: {out_path}")


[OK] 저장 완료: _ml_dl_입문_실습_250804/06_비지도_학습/00_로드맵/6-0_비지도_파이프라인_가이드.ipynb
